In [15]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# โหลดไฟล์
df = pd.read_csv('material_100hz_data.csv')

# การจัดเรียงตัวแปรแบบอัตโนมัติ (ใช้ตำแหน่งคอลัมน์)
X = df.iloc[:, :-1]  # เลือกทุกคอลัมน์ "ยกเว้น" คอลัมน์สุดท้าย (Features)
y = df.iloc[:, -1]   # เลือกเฉพาะ "คอลัมน์สุดท้าย" (Target/Label)

In [18]:
# แบ่งข้อมูลเป็น 80% สำหรับเทรน และ 20% สำหรับทดสอบ
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

# สร้าง XGBoost Regressor
xgb_model = XGBRegressor(random_state=1)
xgb_model.fit(X_train, y_train)

# ทำนาย
y_pred = xgb_model.predict(X_test)

# แสดงผลลัพธ์
print("R2 Score: ", r2_score(y_test, y_pred))
print("MSE: ", mean_squared_error(y_test, y_pred))
print("RMSE: ", mean_squared_error(y_test, y_pred))

R2 Score:  0.8997317151054836
MSE:  3.208815475941383e-05
RMSE:  3.208815475941383e-05


In [25]:
import numpy as np
from scipy.optimize import minimize
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd

# โหลดข้อมูล
df = pd.read_csv('material_100hz_data.csv')
X = df.iloc[:, :4]   # Proportion1, Proportion2, Temp_C, Pressure_bar
y = df.iloc[:, 4:]   # Alpha_1Hz ถึง Alpha_100Hz

# เทรน XGBoost model สำหรับแต่ละ Hz
models = {}
for col in y.columns:
    model = XGBRegressor(random_state=1, n_estimators=100)
    model.fit(X, y[col])
    models[col] = model

# กำหนดค่า output ที่ต้องการ (ตัวอย่าง: ลดลงจาก 0.5 ไป 0.3)
# คุณสามารถแก้ไขตรงนี้ได้ตามต้องการ
target_output = np.linspace(0.1, 0.98, 100)  # 0.5 ที่ 1Hz ลดลงเรื่อยๆ ถึง 0.3 ที่ 100Hz

# หรือถ้าต้องการกำหนดเอง:
# target_output = [0.5, 0.5, 0.5, 0.4, 0.4, 0.4, 0.4, 0.4, 0.3, ...] # ใส่ครบ 100 ค่า

# ฟังก์ชัน objective: หา input ที่ให้ output ใกล้เคียง target มากที่สุด
def objective(x):
    predictions = []
    for i, col in enumerate(y.columns):
        pred = models[col].predict([x])[0]
        predictions.append(pred)
    predictions = np.array(predictions)
    # คำนวณ Mean Squared Error
    mse = np.mean((predictions - target_output) ** 2)
    return mse

# กำหนดขอบเขตของ input (min, max จากข้อมูลจริง)
bounds = [
    (X['Proportion1'].min(), X['Proportion1'].max()),   # Proportion1
    (X['Proportion2'].min(), X['Proportion2'].max()),   # Proportion2  
    (X['Temp_C'].min(), X['Temp_C'].max()),             # Temp_C
    (X['Pressure_bar'].min(), X['Pressure_bar'].max()), # Pressure_bar
]

# เริ่มต้นจากค่าเฉลี่ย
x0 = X.mean().values

# หาค่า optimal
result = minimize(objective, x0, method='L-BFGS-B', bounds=bounds)

# แสดงผลลัพธ์
print("=" * 50)
print("ค่า Input ที่เหมาะสม:")
print(f"  Proportion1:   {result.x[0]:.4f}")
print(f"  Proportion2:   {result.x[1]:.4f}")
print(f"  Temp_C:        {result.x[2]:.4f}")
print(f"  Pressure_bar:  {result.x[3]:.4f}")
print("=" * 50)
print(f"MSE (ความคลาดเคลื่อน): {result.fun:.6f}")

ค่า Input ที่เหมาะสม:
  Proportion1:   51.6667
  Proportion2:   48.3333
  Temp_C:        140.0000
  Pressure_bar:  4.0000
MSE (ความคลาดเคลื่อน): 0.073270
